# Depuis l'ensemble du corpus dans data/corpus_a_traiter et avec la base de donnée corpus.csv crée un corpus étiquetté par genre_auteur_titre dans corpus_genre

In [10]:
import os
import pandas as pd
import shutil

In [5]:
def makecorp(dossier_corpus, dossier_destination, 'Genre_sum'):
    if not os.path.exists(dossier_destination):
        os.makedirs(dossier_destination)

    df = pd.read_csv('data/corpus.csv')
    df = df.dropna(subset=['Genre','Fichier'])

    fichiers_reels = os.listdir(dossier_corpus)

    for index, row in df.iterrows():
        fichier = row['Fichier']
        genre = row['Genre_sum']

        if fichier in fichiers_reels:
            if genre not in ['théâtre', 'poésie']:
                if genre=='Nouvelles':
                    genre='Roman'
                nouveau_nom = f"{genre}_{fichier}"

                shutil.copy(os.path.join(dossier_corpus, fichier), os.path.join(dossier_destination, nouveau_nom))

In [ ]:
dossier_corpus = 'data/corpus_a_traiter'
dossier_destination = 'data/genre'
makecorp(dossier_corpus, dossier_destination, 'Genre')

In [6]:
dossier_corpus = 'data/corpus_a_traiter'
dossier_destination = 'data/genre_sum'
makecorp(dossier_corpus, dossier_destination, 'Genre_sum')

# pour SVM divise en 2 classe le corpus en genre Pamphlet et en genre ALL

In [1]:
path = 'data/genre'
for filename in os.listdir(path):
    if filename.endswith('.txt'):
        genre, author, title = filename[:-4].split('_')  # Retire l'extension ".txt" et divise le nom de fichier en parties
        
        if genre != 'Pamphlet':
            new_filename = f'all_{author}_{title}.txt'  # Préfixe "all_" pour les autres genres
            os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

NameError: name 'os' is not defined

# corpus_genre to corpus author

In [ ]:
path_origin = 'data/genre'
path_out = 'data/author'

if not os.path.isdir(path_out):

    os.mkdir(path_out)
    
for t in os.listdir(path_origin):
    t_path = os.path.join(path_origin,t)
    
    a = '_'.join(t.split('_')[1:])
    a_path = os.path.join(path_out,a)
    print(t_path,'-->',a_path)
    shutil.copy(t_path, a_path)

# corpus_genre sans mot_outils

In [ ]:
import os
import shutil

# Chemins vers les répertoires d'origine et de destination
path_origin = 'data/corpus_genre'
path_out = 'data/corpus_genre_no_stopwords'
stopwordsfile = 'data/stopwords--fr.txt'

# Fonction pour charger la liste des stopwords depuis le fichier
def load_stopwords(stopwordsfile):
    with open(stopwordsfile, 'r', encoding='utf-8') as f:
        stopwords = set(word.strip() for word in f.readlines())
    return stopwords

# Créer le répertoire de destination s'il n'existe pas
if not os.path.exists(path_out):
    os.mkdir(path_out)

# Charger la liste des stopwords
stopwords = load_stopwords(stopwordsfile)

# Parcourir les fichiers dans le répertoire d'origine
for t in os.listdir(path_origin):
    t_path = os.path.join(path_origin, t)
    
    a_path = os.path.join(path_out, t)
    
    # Copier le fichier en supprimant les stopwords
    with open(t_path, 'r', encoding='utf-8') as source_file:
        with open(a_path, 'w', encoding='utf-8') as dest_file:
            for line in source_file:
                # Supprimer les stopwords de chaque ligne du fichier
                words = line.split()
                filtered_words = [word for word in words if word.lower() not in stopwords]
                dest_file.write(' '.join(filtered_words) + '\n')

    print(t_path, '-->', a_path)

# Transformation corpus xml en txt

In [25]:
import os
import xml.etree.ElementTree as ET

def extract_text_from_xml(xml_content):
    tree = ET.ElementTree(ET.fromstring(xml_content))
    
    title = ""
    author = ""
    genre = ""
    
    teiHeader = tree.find(".//teiHeader")
    if teiHeader is not None:
        titleStmt = teiHeader.find(".//titleStmt")
        if titleStmt is not None:
            title = titleStmt.findtext(".//title").replace('_', ' ')
            author_elem = titleStmt.find(".//author")
            if author_elem is not None:
                author = author_elem.get("name", "")
                author = author.replace(',', '').replace('_', ' ')
        
        profileDesc = teiHeader.find(".//profileDesc")
        if profileDesc is not None:
            keywords = profileDesc.findall(".//term")
            if keywords:
                genre = " ".join([keyword.text for keyword in keywords if keyword.text])

                if genre == 'cycles et séries':
                    genre = 'roman'
                elif 'nouvelles' in genre:
                    genre = 'nouvelle'
                elif 'cycles et séries' in genre:
                    genre = genre.replace('cycles et séries', '')
                elif 'fantastique' in genre:
                    genre = genre.replace('fantastique', '')
                    
                elif 'policier' in genre:
                    genre = genre.replace('policier', '')
                if 'récit de voyage' in genre:
                    genre = 'mémoire et bio.'
                if 'journal intime' in genre:
                    genre = 'mémoire et bio.'
                if 'roman' in genre:
                    genre = 'roman'
                if 'mémoires et autobiographie' in genre:
                    genre = 'mémoire et bio.'
                if genre in 'épistolaire':
                    genre = 'mémoire et bio.'
                if genre in ['littérature jeunesse','érotisme','non-fiction','science-fiction','littérature de jeunesse','fantastique']:
                    return "", "", "", ""

            else:
                print("No genre found, skipping text extraction.")
                return "", "", "", ""
    
    text_body = tree.find(".//text//body")
    if text_body is not None:
        paragraphs = text_body.findall(".//p")
        extracted_text = "\n".join([p.text for p in paragraphs if p.text])
        return extracted_text, title, author, genre
    return "", title, author, genre

def process_files(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".xml"):
            input_path = os.path.join(input_folder, filename)
            with open(input_path, "r", encoding="utf-8") as f:
                xml_content = f.read()

            extracted_text, title, author, genre = extract_text_from_xml(xml_content)

            if extracted_text and genre:
                genre = genre.strip()  # Supprime les espaces en début et fin
                file_name = f"{genre}_{author}_{title}.txt"
                file_name = file_name.replace('/','')
                output_path = os.path.join(output_folder, file_name)
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write(extracted_text)
                print(f"Text extracted {output_path}")
            else:
                print(f"No {input_path}")

def main():
    path_in = 'data/2000romans19e20e_xml'
    path_out = 'data/2000romans19e20e'
    
    process_files(path_in, path_out)

if __name__ == "__main__":
    main()


No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1888_Greville-Henry_La-seconde-mere.xml
Text extracted data/2000romans19e20e/roman_Stapleaux Léopold_ Le pendu de la Forêt-Noire (1880) .txt
Text extracted data/2000romans19e20e/nouvelle_Theuriet André_ André Theuriet. Toute seule. Un miracle. Saint Énogat. 3e édition - Edition 3 (1880) .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1855_Ulbach-Louis_L-amour-et-la-mort.xml
No data/2000romans19e20e_xml/1880_Gouraud-Julie_Aller-et-retour.xml
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1900_Roussel-Raymond_Chiquenaude.xml
Text extracted data/2000romans19e20e/mémoire et bio._Aimard Gustave_ Le Guaranis .txt
Text extracted data/2000romans19e20e/roman_Dumas (père) Alexandre_ Les Mohicans de Paris 4 .txt
Text extracted data/2000romans19e20e/roman_Dumas (père) Alexandre_Les Blancs et les Bleus.txt
Text extracted data/2000romans19e20e/nouvelle_Loti Pierre_ Le livre de l

Text extracted data/2000romans19e20e/roman_Delpit Albert_ Jean-nu-pieds, chronique de 1832 (volume 2) .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1911_Barres-Maurice_Le-culte-du-moi_1-Sous-l'œil-des-barbares-nouvelle-edition.xml
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1894_Lesueur-Daniel_Haine-d-amour.xml
Text extracted data/2000romans19e20e/roman_Petitjean de la Rosière Jeanne-Marie_Une mésalliance.txt
Text extracted data/2000romans19e20e/nouvelle_Cim Albert_ Contes et souvenirs de mon pays .txt
Text extracted data/2000romans19e20e/roman_Ponson du Terrail Pierre_ La reine des barricades .txt
Text extracted data/2000romans19e20e/mémoire et bio._Leblanc Maurice_ Le Cercle rouge .txt
Text extracted data/2000romans19e20e/roman_Sand George_Consuelo.txt
Text extracted data/2000romans19e20e/nouvelle_Chavette Eugène_ Nous marions Virginie .txt
Text extracted data/2000romans19e20e/mémoire et bio._Lermina Jules_ Histoires incroyable

Text extracted data/2000romans19e20e/roman_Gondrecourt Aristide de_ Le Pays de la peur, par A. de Gondrecourt - Tome 1 .txt
Text extracted data/2000romans19e20e/roman_Aimard Gustave_ Coeur-de-Panthère .txt
Text extracted data/2000romans19e20e/nouvelle_Leblanc Maurice_ L’Agence Barnett et Cie .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1911_Montesquiou-Fezensac-Robert-de_La-petite-mademoiselle.xml
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1856_Texier-Edmond-Auguste_Amour-et-finance.xml
No data/2000romans19e20e_xml/1854_Dumas-Alexandre_La-jeunesse-de-Pierrot.xml
Text extracted data/2000romans19e20e/mémoire et bio._Verne Jules_Le rayon vert.txt
Text extracted data/2000romans19e20e/mémoire et bio._Toulet Paul Jean_ Les tendres ménages .txt
Text extracted data/2000romans19e20e/roman_France Anatole_ Sur la pierre blanche .txt
Text extracted data/2000romans19e20e/mémoire et bio._Tinseau Léon de_ Ma cousine Pot-au-feu .txt
Text extra

No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1894_Stendhal_Lucien-Leuwen.xml
Text extracted data/2000romans19e20e/roman_France Anatole_ L’anneau d’améthyste .txt
Text extracted data/2000romans19e20e/roman_Arlincourt Charles-Victor Prévost d'_Le Solitaire.txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1847_Sandeau-Jules_Mademoiselle-de-la-Seigliere_Volume-1.xml
Text extracted data/2000romans19e20e/roman_Le Rouge Gustave_ La Reine des éléphants .txt
Text extracted data/2000romans19e20e/nouvelle_Leblanc Maurice_ Contes .txt
Text extracted data/2000romans19e20e/roman_Proust Marcel_Le Côté de Guermantes (À la recherche du temps perdu).txt
Text extracted data/2000romans19e20e/roman_Verne Jules_Les Forceurs de blocus.txt
Text extracted data/2000romans19e20e/mémoire et bio._Ponson du Terrail Pierre_ La Baronne trépassée .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1876_Sand-George_Horace.xml
No genre found, sk

Text extracted data/2000romans19e20e/roman_Proust Marcel_Sodome et Gomorrhe — À la recherche du temps perdu.txt
Text extracted data/2000romans19e20e/nouvelle_Balzac Honoré de_Madame Firmiani.txt
Text extracted data/2000romans19e20e/nouvelle_Eyma Xavier_ Le Médaillon .txt
No data/2000romans19e20e_xml/1889_Dombre-Roger_Folla.xml
Text extracted data/2000romans19e20e/roman_Giberton Theodore_La fille du vieux chanteur.txt
Text extracted data/2000romans19e20e/mémoire et bio._Lachèse Marthe_ Le Mariage de Renée, par Mlle Marthe Lachèse (1879) .txt
Text extracted data/2000romans19e20e/mémoire et bio._Vidocq Eugène-François_ Mémoires de Vidocq - Tome II .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1874_Flaubert-Gustave_La-tentation-de-saint-Antoine.xml
Text extracted data/2000romans19e20e/roman_Halt Marie Robert_ Histoire d'un petit homme .txt
Text extracted data/2000romans19e20e/mémoire et bio._Nadaud Marcel_ Chignole (la guerre aérienne) .txt
Text extracted data

...

Text extracted data/2000romans19e20e/roman_Dumas (père) Alexandre_ Olympe de Clèves I .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1875_Noel-Eugene_Memoires-d-un-imbecile.xml
Text extracted data/2000romans19e20e/mémoire et bio._Cadol Édouard_ Son Excellence Satinette (affaires étrangères), par Édouard Cadol (1882) .txt
Text extracted data/2000romans19e20e/roman_Dumas (père) Alexandre_La San Felice.txt
Text extracted data/2000romans19e20e/nouvelle_Maupassant Guy de_Miss Harriet.txt
Text extracted data/2000romans19e20e/mémoire et bio._Dumas (père) Alexandre_ Black .txt
Text extracted data/2000romans19e20e/mémoire et bio._Zaccone Pierre_ Éric le mendiant .txt
Text extracted data/2000romans19e20e/roman_Le Rouge Gustave_ Les Aventures de Todd Marvel, détective milliardaire .txt
Text extracted data/2000romans19e20e/roman_Petitjean de la Rosière Jeanne-Marie_Le mystère de Ker Even.txt
Text extracted data/2000romans19e20e/mémoire et bio._Sand George_Monsieur Sylv

Text extracted data/2000romans19e20e/mémoire et bio._Galopin Arnould_ La ténébreuse affaire de Green-Park .txt
Text extracted data/2000romans19e20e/mémoire et bio._Loti Pierre_ La Galilée .txt
Text extracted data/2000romans19e20e/nouvelle_Witt Henriette de_ Scènes historiques (2ème série) .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1910_Lesueur-Daniel_Chacune-son-Reve.xml
Text extracted data/2000romans19e20e/nouvelle_Normand Jacques_ Le Monde où nous sommes, par Jacques Normand (1884) .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1869_Sorr-Angelo-de_Fantome-de-la-rue-de-Venise.xml
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1868_Perret-Paul_La-Parisienne.xml
Text extracted data/2000romans19e20e/nouvelle_Balzac Honoré de_Adieu.txt
No data/2000romans19e20e_xml/1844_Chateaubriand-François-Rene-de_Vie-de-Rance.xml
Text extracted data/2000romans19e20e/roman_Révillon Tony_ Noémi, par Tony Révillon (1878)

Text extracted data/2000romans19e20e/mémoire et bio._Gaboriau Émile_ Les Esclaves de Paris - Tome I .txt
No data/2000romans19e20e_xml/1875_Mirabeau-Marie-Le-Harivel-de-Gonneville_Ctesse-de-Mirabeau-Jane-et-Germaine-Voyages-d-un-capitaine.xml
Text extracted data/2000romans19e20e/mémoire et bio._Dondel Du Faouëdic Noémie_ Voyages loin de ma chambre tome 1 .txt
Text extracted data/2000romans19e20e/mémoire et bio._Ponson du Terrail Pierre_ Mémoires d'un gendarme .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1907_Gide-Andre_Le-Retour-de-l-enfant-prodigue.xml
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1866_Feuillet-Octave_Monsieur-de-Camors.xml
No data/2000romans19e20e_xml/1910_Cim-Albert_Le-petit-Leveille.xml
Text extracted data/2000romans19e20e/mémoire et bio._Chateaubriand François-René de_ Mémoires d'Outre-Tombe, Tome 3 .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1897_Roussel-Raymond_La-doublure.xml

No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1881_Malot-Hector_Une-femme-d-argent.xml
Text extracted data/2000romans19e20e/roman_Buet Charles_ Philippe-Monsieur .txt
No data/2000romans19e20e_xml/1877_Fleuriot-Zenaide_Un-enfant-gate.xml
Text extracted data/2000romans19e20e/nouvelle_Legouvé Ernest_ Nos filles et nos fils, scènes et études de famille .txt
Text extracted data/2000romans19e20e/nouvelle_Bellaud E. de_ Ces pauvres filles ! .txt
Text extracted data/2000romans19e20e/nouvelle_Leblanc Maurice_ Le Cabochon d'émeraude .txt
Text extracted data/2000romans19e20e/roman_Féval (père) Paul_ Le Fils du diable – Tome II .txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1886_Bourget-Paul_Andre-Cornelis.xml
Text extracted data/2000romans19e20e/mémoire et bio._Ponson du Terrail Pierre_ Les Nuits du quartier Bréda. Juliette .txt
Text extracted data/2000romans19e20e/roman_Zévaco Michel_ Le Fils de Pardaillan: grand roman de cape et d'épée, t. 2

Text extracted data/2000romans19e20e/roman_Petitjean de la Rosière Jeanne-Marie_La maison du lis.txt
Text extracted data/2000romans19e20e/roman_Zola Émile_Nana.txt
Text extracted data/2000romans19e20e/roman_Bouvier Alexis_ La Grande Iza .txt
Text extracted data/2000romans19e20e/littérature jeunesse science-fiction_Robida Albert_ Un chalet dans les airs .txt
Text extracted data/2000romans19e20e/roman_Zola Émile_L'assommoir.txt
No genre found, skipping text extraction.
No data/2000romans19e20e_xml/1833_Musset-Alfred-de_Gamiani-ou-Deux-Nuits-d-Exces.xml
Text extracted data/2000romans19e20e/roman_Féval (père) Paul_ La Louve - Tome II - Valentine de Rohan .txt
Text extracted data/2000romans19e20e/roman_France Anatole_ Les Dieux ont soif .txt
Text extracted data/2000romans19e20e/roman_Tissot Marcel_ La Veuve d'Attila (1881) .txt
Text extracted data/2000romans19e20e/roman_Balzac Honoré de_Béatrix.txt
Text extracted data/2000romans19e20e/roman_Beaufort d'Hautpoul Anne-Marie de_ Le Page et la r

In [29]:
# Supprimer les genres avec moins de 25 textes
df = df_result_mots
genres_a_supprimer = df[df['Nombre'] < 25]['Genre']
for genre in genres_a_supprimer:
    fichiers_genre = [fichier for fichier in os.listdir(dossier) if fichier.startswith(genre + '_')]
    for fichier in fichiers_genre:
        chemin_fichier = os.path.join(dossier, fichier)
        os.remove(chemin_fichier)

print("Fichiers des genres avec moins de 25 textes supprimés.")


Fichiers des genres avec moins de 25 textes supprimés.


# Créer coprus équilibré

In [1]:
import os
import pandas as pd

def analyze_corpus(dossier, mode='caracteres'):
    """
    Analyse les fichiers d'un dossier et crée un DataFrame contenant les statistiques.
    
    :param dossier: Le chemin vers le dossier contenant les fichiers à analyser.
    :param mode: Le mode d'analyse, soit 'caracteres' (par défaut) soit 'mots'.
    :return: Le DataFrame contenant les statistiques.
    """
    # Dictionnaires pour stocker les statistiques
    compte_genres = {}
    somme_longueurs_genres = {}
    min_longueurs_genres = {}
    max_longueurs_genres = {}
    moyennes_longueurs_genres = {}

    # Parcours de tous les fichiers dans le dossier
    for nom_fichier in os.listdir(dossier):
        if nom_fichier.endswith('.txt'):
            # Extraction du genre à partir du nom de fichier
            genre = nom_fichier.split('_')[0]
            
            # Lecture du contenu du fichier
            chemin_fichier = os.path.join(dossier, nom_fichier)
            with open(chemin_fichier, 'r') as f:
                contenu = f.read()
                if mode == 'caracteres':
                    longueur = len(contenu)
                elif mode == 'mots':
                    longueur = len(contenu.split())
                else:
                    raise ValueError("Le mode doit être 'caracteres' ou 'mots'")
            
            # Mise à jour des dictionnaires pour le genre
            if genre in compte_genres:
                compte_genres[genre] += 1
                somme_longueurs_genres[genre] += longueur
                min_longueurs_genres[genre] = min(min_longueurs_genres[genre], longueur)
                max_longueurs_genres[genre] = max(max_longueurs_genres[genre], longueur)
                moyennes_longueurs_genres[genre] = somme_longueurs_genres[genre] / compte_genres[genre]
            else:
                compte_genres[genre] = 1
                somme_longueurs_genres[genre] = longueur
                min_longueurs_genres[genre] = longueur
                max_longueurs_genres[genre] = longueur
                moyennes_longueurs_genres[genre] = longueur

    # Création d'un DataFrame pandas
    donnees = {'Genre': list(compte_genres.keys()),
               'Nombre': list(compte_genres.values()),
               'Somme': list(somme_longueurs_genres.values()),
               'Min': list(min_longueurs_genres.values()),
               'Max': list(max_longueurs_genres.values()),
               'Moyenne': list(moyennes_longueurs_genres.values())}
    df = pd.DataFrame(donnees)

    # Formattage des colonnes pour afficher les nombres sous forme standard
    pd.set_option('display.float_format', '{:.2f}'.format)

    # Style Veridis sur toutes les colonnes (sauf 'Genre')
    style = df.style.background_gradient(cmap='viridis', subset=df.columns.difference(['Genre']))

    # Affichage du style du DataFrame
    print(mode)
    display(style)
    return df


In [30]:
dossier = 'data/2000romans19e20e/'
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,nouvelle,260,10460586,844,137301,40233.023077
1,roman,772,64706248,681,570625,83816.383420
2,mémoire et bio.,369,25913337,1399,314989,70225.845528


In [15]:
dossier = 'data/genre_sum_1/'
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,pamphlet,48,2931562,2356,263090,61074.208333
1,essai,37,2527073,2108,819600,68299.270270
2,roman,78,5702861,1450,284630,73113.602564
3,article,25,473482,570,114731,18939.280000
4,mémoire et bio.,13,757025,6043,88988,58232.692308


In [12]:
dossier = 'data/pamphlet/'
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,pamphlet,48,2931562,2356,263090,61074.208333


In [32]:
dossier = 'data/corpus_balanced_optimized/'
#df_result_caracteres = analyze_corpus(dossier, mode='caracteres')
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,nouvelle,141,2935918,844,41442,20822.113475
1,pamphlet,48,2931562,2356,263090,61074.208333
2,roman,109,2939952,681,38207,26972.036697
3,mémoire et bio.,92,2951061,1399,45185,32076.750000


# échantillonage manuel

In [2]:
import os

def echantillonage(input_folder, target_length):
    output_folder = f"{input_folder}_{target_length}-samples"
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_folder, exist_ok=True)

    # Parcourir les fichiers du dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_folder, filename)

            with open(input_path, "r", encoding="utf-8") as input_file:
                text = input_file.read()
                words = text.split()
                total_words = len(words)

                num_samples = total_words // target_length  # Calculer le nombre d'échantillons entiers
                remaining_words = total_words % target_length  # Mots restants

                if num_samples == 0:
                    # Si le texte est plus court que la taille cible, ne faire qu'un échantillon
                    sampled_text = " ".join(words)
                    output_filename = f"{filename.replace('.txt', '')}_sample_1.txt"
                    output_path = os.path.join(output_folder, output_filename)

                    with open(output_path, "w", encoding="utf-8") as output_file:
                        output_file.write(sampled_text)
                else:
                    for i in range(num_samples - 1):
                        start_idx = i * target_length
                        end_idx = (i + 1) * target_length
                        sampled_words = words[start_idx:end_idx]
                        sampled_text = " ".join(sampled_words)

                        output_filename = f"{filename.replace('.txt', '')}_sample_{i+1}.txt"
                        output_path = os.path.join(output_folder, output_filename)

                        with open(output_path, "w", encoding="utf-8") as output_file:
                            output_file.write(sampled_text)

                    # Dernier échantillon avec les mots restants intégrés
                    start_idx = (num_samples - 1) * target_length
                    end_idx = total_words
                    sampled_words = words[start_idx:end_idx]
                    sampled_text = " ".join(sampled_words)

                    last_output_filename = f"{filename.replace('.txt', '')}_sample_{num_samples}.txt"
                    last_output_path = os.path.join(output_folder, last_output_filename)

                    with open(last_output_path, "w", encoding="utf-8") as last_output_file:
                        last_output_file.write(sampled_text)


In [ ]:
import os

def echantillonage(input_folder, target_length):
    output_folder = f"{input_folder}_{target_length}-samples"
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_folder, exist_ok=True)

    # Parcourir les fichiers du dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_folder, filename)

            with open(input_path, "r", encoding="utf-8") as input_file:
                text = input_file.read()
                words = text.split()
                total_words = len(words)

                num_samples = total_words // target_length  # Calculer le nombre d'échantillons entiers
                remaining_words = total_words % target_length  # Mots restants

                if num_samples == 0:
                    # Si le texte est plus court que la taille cible, ne faire qu'un échantillon
                    sampled_text = " ".join(words)
                    output_filename = f"{filename.replace('.txt', '')}_sample_1.txt"
                    output_path = os.path.join(output_folder, output_filename)

                    with open(output_path, "w", encoding="utf-8") as output_file:
                        output_file.write(sampled_text)
                else:
                    for i in range(num_samples - 1):
                        start_idx = i * target_length
                        end_idx = (i + 1) * target_length
                        sampled_words = words[start_idx:end_idx]
                        sampled_text = " ".join(sampled_words)

                        output_filename = f"{filename.replace('.txt', '')}_sample_{i+1}.txt"
                        output_path = os.path.join(output_folder, output_filename)

                        with open(output_path, "w", encoding="utf-8") as output_file:
                            output_file.write(sampled_text)

                    # Dernier échantillon avec les mots restants intégrés
                    last_sample = words[(num_samples) * target_length:total_words]
                    print(len(last_sample))
                    if len(last_sample)>=6500:
                        last_sample_text = " ".join(last_sample)
                        
                        sampled_text = " ".join(words[(num_samples - 1) * target_length:(num_samples) * target_length])
                        
                        last_output_filename = f"{filename.replace('.txt', '')}_sample_{num_samples-1}.txt"
                        last_output_path = os.path.join(output_folder, last_output_filename)
                        with open(last_output_path, "w", encoding="utf-8") as last_output_file:
                            last_output_file.write(sampled_text)
                        
                        last_output_filename = f"{filename.replace('.txt', '')}_sample_{num_samples}.txt"
                        last_output_path = os.path.join(output_folder, last_output_filename)
                        with open(last_output_path, "w", encoding="utf-8") as last_output_file:
                            last_output_file.write(last_sample_text)
                        
                        
                        
                    else:    
                        start_idx = (num_samples - 1) * target_length
                        end_idx = total_words
                        sampled_words = words[start_idx:end_idx]
                        sampled_text = " ".join(sampled_words)

                        last_output_filename = f"{filename.replace('.txt', '')}_sample_{num_samples}.txt"
                        last_output_path = os.path.join(output_folder, last_output_filename)

                        with open(last_output_path, "w", encoding="utf-8") as last_output_file:
                            last_output_file.write(sampled_text)

In [12]:
target_length = 10000
input_folder = "data/genre-motifs"
echantillonage(input_folder, target_length)

In [13]:
dossier = 'data/genre-motifs'
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,essai,37,2802448,2331,908259,75741.837838
1,roman,78,6414840,1566,321964,82241.538462
2,pamphlet,46,2978823,2643,285137,64757.021739


In [14]:
dossier = 'data/genre-motifs_10000-samples'
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,roman,612,6414840,1566,19843,10481.764706
1,essai,270,2802448,2331,19042,10379.437037
2,pamphlet,284,2978823,2643,19850,10488.813380


# sur corpus_genre_sum

In [63]:
dossier = "data/genre_sum/"
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,pamphlet,48,2931562,2356,263090,61074.208333
1,essai,37,2527073,2108,819600,68299.270270
2,roman,63,5106848,1450,284630,81061.079365
3,article,25,473482,570,114731,18939.280000
4,mémoire et bio.,13,757025,6043,88988,58232.692308
5,nouvelle,15,596013,2064,153668,39734.200000


In [17]:
target_length = 10000
input_folder = "data/genre_sum_1"
echantillonage(input_folder, target_length)

In [18]:
dossier = "data/genre_sum_1/"
df_result_mots = analyze_corpus(dossier, mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,pamphlet,48,2931562,2356,263090,61074.208333
1,essai,37,2527073,2108,819600,68299.270270
2,roman,78,5702861,1450,284630,73113.602564
3,article,25,473482,570,114731,18939.280000
4,mémoire et bio.,13,757025,6043,88988,58232.692308


In [19]:

df_result_mots = analyze_corpus('data/genre_sum_1_10000-samples', mode='mots')

mots


,Genre,Nombre,Somme,Min,Max,Moyenne
0,roman,542,5702861,1450,19967,10521.883764
1,essai,239,2527073,2108,19600,10573.527197
2,pamphlet,277,2931562,2356,19970,10583.256318
3,article,56,473482,570,18900,8455.035714
4,mémoire et bio.,70,757025,6043,18988,10814.642857


In [25]:
import pandas as pd

# Lire le fichier CSV
data = pd.read_csv('misattributions.csv')

# Générer le tableau LaTeX
latex_table = '\\begin{lontable}\n'
latex_table += '    \\centering\n'
latex_table += '        \\toprule\n'
latex_table += '             & id & Vrai & prédiction \\\\\n'
latex_table += '        \\hline\n'


for index, row in data.iterrows():
    latex_table += f'        {row["id"]} & {row["True"]} & {row["Pred"]} \\\\\n'
    latex_table += '        \\hline\n'

latex_table += '    \\caption{Table des mauvaises attributions SVM corpus 1}\n'
latex_table += '    \\label{tab:misattributions-corpus1}\n'
latex_table += '\\end{longtable}'

print(latex_table)

\begin{lontable}
    \centering
        \toprule
             & id & Vrai & prédiction \\
        \hline
        article_Barrès_Discours de réception à l’Académie française_sample_1.txt & article & roman \\
        \hline
        article_Barrès_La Querelle des nationalistes et des cosmopolites_sample_1.txt & article & roman \\
        \hline
        article_Barrès_La Terre et les morts_sample_1.txt & article & pamphlet \\
        \hline
        article_Barrès_Le Bi-centenaire de Jean-Jacques Rousseau_sample_1.txt & article & pamphlet \\
        \hline
        article_Barrès_Les tâches d’encre_sample_1.txt & article & roman \\
        \hline
        article_Barrès_Les Traits éternels de la France_sample_1.txt & article & roman \\
        \hline
        article_d’Axa_Endehors_sample_1.txt & article & pamphlet \\
        \hline
        article_d’Axa_Les Feuilles de Zo d’Axa_sample_3.txt & article & pamphlet \\
        \hline
        article_Mirbeau_articles Dreyfus_sample_1.txt & article 